In [1]:
import os
os.chdir('D:\columbia qmss\MDS\course_content21\Exercises\HW02')

In [2]:
import pandas as pd
df = pd.read_csv("U.S._Chronic_Disease_Indicators__CDI_.csv")

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Selection of Data and Reshaping the Data

In [3]:
# Glimpse the data
df.columns

Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'DataSource',
       'Topic', 'Question', 'Response', 'DataValueUnit', 'DataValueType',
       'DataValue', 'DataValueAlt', 'DataValueFootnoteSymbol',
       'DatavalueFootnote', 'LowConfidenceLimit', 'HighConfidenceLimit',
       'StratificationCategory1', 'Stratification1', 'StratificationCategory2',
       'Stratification2', 'StratificationCategory3', 'Stratification3',
       'GeoLocation', 'ResponseID', 'LocationID', 'TopicID', 'QuestionID',
       'DataValueTypeID', 'StratificationCategoryID1', 'StratificationID1',
       'StratificationCategoryID2', 'StratificationID2',
       'StratificationCategoryID3', 'StratificationID3'],
      dtype='object')

In [4]:
df['DataValueType'].unique()

array(['Average Annual Number', 'Number', 'Crude Prevalence',
       'Age-adjusted Prevalence', 'Mean', 'Age-adjusted Mean', 'Yes/No',
       'Percent', 'US Dollars', 'Per capita alcohol consumption',
       'Prevalence', 'Age-adjusted Rate', 'Crude Rate',
       'Average Annual Crude Rate', 'Average Annual Age-adjusted Rate',
       'Adjusted by age, sex, race and ethnicity', 'Median',
       'Local control of the regulation of alcohol outlet density',
       'Commercial host (dram shop) liability status for alcohol service'],
      dtype=object)

In [5]:
df['DataValueUnit'].unique()

array([nan, 'Years', '%', 'Number', '$', 'gallons', 'cases per 1,000',
       'per 100,000', 'cases per 1,000,000', 'cases per 100,000',
       'cases per 10,000', 'pack sales per capita',
       'per 100,000 residents'], dtype=object)

In [6]:
df["Topic"].unique()

array(['Cancer', 'Alcohol', 'Asthma', 'Chronic Kidney Disease',
       'Cardiovascular Disease', 'Overarching Conditions', 'Diabetes',
       'Tobacco', 'Arthritis', 'Chronic Obstructive Pulmonary Disease',
       'Mental Health', 'Nutrition, Physical Activity, and Weight Status',
       'Oral Health', 'Older Adults', 'Disability', 'Reproductive Health',
       'Immunization'], dtype=object)

In [7]:
df["StratificationCategory1"].unique()

array(['Gender', 'Overall', 'Race/Ethnicity'], dtype=object)

In [15]:
df["Stratification1"].unique()

array(['Female', 'Overall', 'Male', 'Hispanic', 'White, non-Hispanic',
       'Black, non-Hispanic', 'Multiracial, non-Hispanic',
       'Asian or Pacific Islander', 'American Indian or Alaska Native',
       'Asian, non-Hispanic', 'Other, non-Hispanic'], dtype=object)

In [8]:
df["Question"][df['Question'].str.startswith("Binge")].unique()

array(['Binge drinking prevalence among youth',
       'Binge drinking prevalence among adults aged >= 18 years',
       'Binge drinking prevalence among women aged 18-44 years',
       'Binge drinking frequency among adults aged >= 18 years who binge drink',
       'Binge drinking intensity among adults aged >= 18 years who binge drink'],
      dtype=object)

The data contains lots of indicators and is in a long format format. 

1. Remove all columns you do not need for the analysis (All done in Python, of course. No Excel acrobatics.). We are interested in two sets of variables. Select the following variables and remove all others:  
    a) **Binge Drinking**:
    _Binge drinking prevalence among adults aged >= 18 years_, Crude Prevalence in Percent. 
    We would like to obtain this variable for the overall population, as well separately for _females_ and _males_.\
    b) **Poverty**:
    _Poverty, Crude Prevalence in Percent_. We only want the overall poverty prevalence to make things a bit easier.

In [9]:
# Flitering
df_bp = df.query('DataValueType == "Crude Prevalence" \
                 & DataValueUnit == "%" \
                 & (Question == "Binge drinking prevalence among adults aged >= 18 years" \
                 & StratificationCategory1 == ["Overall", "Gender"]) \
                 or (Question == "Poverty" \
                 & StratificationCategory1 =="Overall") \
                 ')[
    ['YearStart', 'LocationAbbr', 'LocationDesc', 
     'Question', 'DataValueAlt', 
     'StratificationCategory1', 'Stratification1']
]
print(df_bp.shape)
df_bp.head()

(1962, 7)


,YearStart,LocationAbbr,LocationDesc,Question,DataValueAlt,StratificationCategory1,Stratification1
340,2015,AL,Alabama,Poverty,18.5,Overall,Overall
369,2014,TX,Texas,Poverty,17.2,Overall,Overall
379,2011,NM,New Mexico,Poverty,21.5,Overall,Overall
473,2011,WY,Wyoming,Poverty,11.3,Overall,Overall
489,2011,ND,North Dakota,Poverty,12.2,Overall,Overall


2. Convert the dataset to a wide format data set using the commands from the `pandas` package.

In [20]:
df_bp_wide = df_bp.pivot(
    index= ['LocationDesc','LocationAbbr','YearStart'],
    columns= ['Question','Stratification1'],
    values= 'DataValueAlt')
df_bp_wide.columns = list(map("_".join, df_bp_wide.columns))
df_bp_wide.head()

Poverty_Overall  \
LocationDesc LocationAbbr YearStart                    
Alabama      AL           2010                  19.0   
                          2011                  19.0   
                          2012                  19.0   
                          2013                  18.7   
                          2014                  19.3   

                                     Binge drinking prevalence among adults aged >= 18 years_Female  \
LocationDesc LocationAbbr YearStart                                                                   
Alabama      AL           2010                                                     NaN                
                          2011                                                     7.9                
                          2012                                                     7.0                
                          2013                                                     6.5                
                          2014                                                     6.8                

                                     Binge drinking prevalence among adults aged >= 18 years_Male  \
LocationDesc LocationAbbr YearStart                                                                 
Alabama      AL           2010                                                     NaN              
                          2011                                                    20.2              
                          2012                                                    18.2              
                          2013                                                    16.3              
                          2014                                                    17.9              

                                     Binge drinking prevalence among adults aged >= 18 years_Overall  
LocationDesc LocationAbbr YearStart                                                                   
Alabama      AL           2010                                                     NaN                
                          2011                                                    13.7                
                          2012                                                    12.3                
                          2013                                                    11.2                
                          2014                                                    12.0

3. Rename the variables to follow the format below.

    Your dataset should now be in a wide state-year format with the following variables:  
      - `state`: Name of the State  
      - `stateabb`: State Abbreviation  
      - `year`: year of observation  
      - `binge_all`: Binge drinking prevalence among _all_ adults aged >= 18 years  
      - `binge_male`: Binge drinking prevalence among _male_ adults aged >= 18 years  
      - `binge_female`: Binge drinking prevalence among _female_ adults aged >= 18 years  
      - `poverty`: Poverty, Crude Prevalence in Percent  
    
    Provide an overview of the dataset by printing its size (using the `shape` command) and some summary statistics (using the `describe` command).
      
    Save the cleaned dataset as `binge_clean.csv`. That file should be included in the uploaded files for your homework submission.

In [27]:
binge_clean = df_bp_wide.reset_index()
binge_clean = binge_clean.rename(columns = {'LocationDesc':'state',
                                          'LocationAbbr': 'stateabb',
                                          'YearStart' : 'year',
                                          'Binge drinking prevalence among adults aged >= 18 years_Female': 'binge_female',
                                          'Binge drinking prevalence among adults aged >= 18 years_Male': 'binge_male',
                                          'Binge drinking prevalence among adults aged >= 18 years_Overall': 'binge_all',
                                          'Poverty_Overall': 'poverty'})

binge_clean.head()

,state,stateabb,year,poverty,binge_female,binge_male,binge_all
0,Alabama,AL,2010,19.0,NaN,NaN,NaN
1,Alabama,AL,2011,19.0,7.9,20.2,13.7
2,Alabama,AL,2012,19.0,7.0,18.2,12.3
3,Alabama,AL,2013,18.7,6.5,16.3,11.2
4,Alabama,AL,2014,19.3,6.8,17.9,12.0


In [29]:
binge_clean.shape

(548, 7)

In [31]:
binge_clean.describe()

,year,poverty,binge_female,binge_male,binge_all
count,548.000000,477.000000,486.000000,486.000000,486.000000
mean,2014.516423,14.935220,11.761111,22.474280,16.968313
std,2.867218,5.227848,2.838002,3.729218,3.116518
min,2010.000000,7.300000,5.000000,13.100000,9.600000
25%,2012.000000,11.800000,9.900000,20.200000,15.100000
50%,2015.000000,14.000000,11.800000,22.100000,16.800000
75%,2017.000000,16.900000,13.200000,24.300000,18.700000
max,2019.000000,46.200000,22.300000,33.800000,27.200000


In [32]:
binge_clean.to_csv('binge_clean.csv', index = False)

## Data Transformation and Summary Results

4. Produce a table that shows the overall, female, and male binge drinking prevalences across U.S. States in the most recent year of data for the Top 10 binge drinking states (i.e. the ones with the highest prevalence in the overall population). Use the relevant `pandas` commands to select the right variables, sort the data, and filter the data frame.

In [33]:
top10_binge =  binge_clean.query('year==2019').drop('poverty',axis=1).nlargest(10,'binge_all',keep='all')
#top10_binge =  binge_clean[binge_clean['year']==2019].drop('poverty',axis=1).nlargest(10,'binge_all',keep='all')
top10_binge

,state,stateabb,year,binge_female,binge_male,binge_all
89,District of Columbia,DC,2019,22.3,27.8,24.9
358,North Dakota,ND,2019,14.9,29.2,22.2
537,Wisconsin,WI,2019,16.5,27.7,21.9
168,Iowa,IA,2019,16.5,27.0,21.6
438,South Dakota,SD,2019,16.1,26.4,21.2
288,Nebraska,NE,2019,15.7,26.1,20.9
248,Minnesota,MN,2019,16.3,25.3,20.7
278,Montana,MT,2019,14.7,26.7,20.7
148,Illinois,IL,2019,14.4,25.8,19.9
228,Massachusetts,MA,2019,15.4,23.2,19.1


5. Calculate the average annual growth rates (in percent) of overall binge drinking across states for the years the data is available. One way to get these growth rates, is to group the data by state (`groupby`) and use the `first()` and `last()` commands to get the first and last non-NA percentage followed by dividing the calculated percentage increase by the number of years data is available for. Alternatively, you could use the `pct_change` function to help you out. Provide a table of the _5 states with the largest increases_ and the _5 states with the largest decreases_ in binge drinking prevalence over the time period. 

In [49]:
binge_clean_copy = binge_clean[["state","year","binge_all"]].dropna()
binge_clean_copy.head()

,state,year,binge_all
1,Alabama,2011,13.7
2,Alabama,2012,12.3
3,Alabama,2013,11.2
4,Alabama,2014,12.0
5,Alabama,2015,11.6


In [63]:
avg_binge_clean_first = binge_clean_copy.groupby('state').first().rename(columns={"year":"first_year","binge_all":"binge_all_first"})
avg_binge_clean_last = binge_clean_copy.groupby('state').last().rename(columns={"year":"last_year","binge_all":"binge_all_last"})
avg_binge_clean = pd.merge(avg_binge_clean_first,avg_binge_clean_last, how='left',on='state')
avg_binge_clean['average_annual_growth_rates'] = (avg_binge_clean['binge_all_last']-avg_binge_clean['binge_all_first'])/(avg_binge_clean['last_year']-avg_binge_clean['first_year'])
avg_binge_clean

,first_year,binge_all_first,last_year,binge_all_last,average_annual_growth_rates
state,,,,,
Alabama,2011,13.7,2019,12.4,-0.162500
Alaska,2011,20.8,2019,15.9,-0.612500
Arizona,2011,17.6,2019,15.1,-0.312500
Arkansas,2011,14.1,2019,13.7,-0.050000
California,2011,18.6,2019,16.8,-0.225000
Colorado,2011,20.1,2019,18.2,-0.237500
Connecticut,2011,17.9,2019,15.6,-0.287500
Delaware,2011,20.3,2019,17.2,-0.387500
District of Columbia,2011,25.0,2019,24.9,-0.012500


In [65]:
avg_binge_clean.nlargest(5,'average_annual_growth_rates',keep='all')

,first_year,binge_all_first,last_year,binge_all_last,average_annual_growth_rates
state,,,,,
Tennessee,2011,10.0,2019,14.4,0.5500
Louisiana,2011,16.1,2019,18.9,0.3500
West Virginia,2011,10.1,2019,12.4,0.2875
Oregon,2011,16.4,2019,17.5,0.1375
South Carolina,2011,15.4,2019,15.9,0.0625


In [67]:
avg_binge_clean.nsmallest(5,'average_annual_growth_rates',keep='all')

,first_year,binge_all_first,last_year,binge_all_last,average_annual_growth_rates
state,,,,,
New Jersey,2011,18.2,2018,13.5,-0.671429
Alaska,2011,20.8,2019,15.9,-0.612500
Maryland,2011,18.0,2019,13.7,-0.537500
Hawaii,2011,21.5,2019,17.3,-0.525000
Oklahoma,2011,16.5,2019,12.7,-0.475000
